In [ ]:
import numpy as np 
import pandas as pd 
import itertools 
import smtplib
import perc
import default
import sys
from collections import defaultdict
import gzip # use compressed data files
import copy, operator, optparse, sys, os
import re 
import time

feat_vec = {}
tagset = []
train_data = []

def perc_train(train_data, tagset, numepochs):
    feat_vec = defaultdict(int)
    t1=time.time()
    FEATURE_VEC_SENT=[] 
    FEATURE_DIC={}
    default_tag=tagset[0]
    WEIGHT_VECTOR=[]
    e=0
    for epoch in range(numepochs) : 
        print("epoch {}".format(e))
        MISTAKES=0
        sent_ind=0
        ## ONE SENTENCE PARSING 
        epoch_start=time.time()
        
        for (sent_labels,sent_features) in train_data : 
            
            #print("sentence=",sent_labels)
            if epoch==0 : 
                FEATURE_DIC={}
            elif epoch>0: 
                FEATURE_DIC=WEIGHT_VECTOR[sent_ind]
                     
            argmax_tag=perc.perc_test(FEATURE_DIC, sent_labels, sent_features, tagset, default_tag)   
                           
            labels = copy.deepcopy(sent_labels)
            labels.insert(0, '_B-1 _B-1 _B-1')
            labels.insert(0, '_B-2 _B-2 _B-2') # first two 'words' are _B-2 _B-1
            labels.append('_B+1 _B+1 _B+1')
            labels.append('_B+2 _B+2 _B+2') # last two 'words' are _B+1 _B+2  
            N = len(labels)   
            
            SUM=0
            ## for each word chunk
            feat_index=0
            j=0
            for i in range(2, N-2):
                
                (feat_index, feats) = perc.feats_for_word(feat_index, sent_features)  
               # print(feats)
                fields = labels[i].split()    
               
                word = fields[0]
                actual_tag=fields[2]    
               
        
                #print("word_index = {}, word = {}, actual_tag = {}, argmax_tag= {}".format(i,word,actual_tag,argmax_tag[i])
                amax_tag=argmax_tag[j]
                
                local_keys=list(FEATURE_DIC.keys())
                feat_list=[(x,actual_tag,amax_tag) for x in feats]
              
                if actual_tag!= amax_tag : 
                    MISTAKES+=1
                
                ## UPDATE THE WEIGHTS HERE.
                               
                for (f,b,c) in feat_list :  
                    
                    if b!=c : 
                        SUM+=1
                        if (f,b) in local_keys : 
                            FEATURE_DIC[f,b]+=1
                        else :
                            FEATURE_DIC[f,b]=1
                    
                        if (f,c) in local_keys : 
                            FEATURE_DIC[f,c]-=1
                        else : 
                            FEATURE_DIC[f,c]=-1
                        
                j=j+1                 
                      
            if epoch == 0 : 
                WEIGHT_VECTOR.append(dict(list(FEATURE_DIC.items())))   
            elif epoch>0 :
                OLDER_FEATURE_DIC=WEIGHT_VECTOR[sent_ind]
                CURRENT_FEATURE_DIC=FEATURE_DIC
                SUM_FEATURE_DIC={}

                
                for a in OLDER_FEATURE_DIC.keys() : 
                 
                    SUM_FEATURE_DIC[a]=OLDER_FEATURE_DIC[a]+CURRENT_FEATURE_DIC[a]
                    
                
            
                WEIGHT_VECTOR[sent_ind]=SUM_FEATURE_DIC.copy()
                
                
            sent_ind=sent_ind+1  
            
        print('Num of Mistakes=',MISTAKES)   
        print('epoch {} ended in {}'.format(e,time.time()-epoch_start))
        e=e+1
        
        
    print("Training finished in ",time.time()-t1)
    return WEIGHT_VECTOR

tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)

#train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)

#F={} 
#TESTING
F= defaultdict(int)

for i in range(len(feat_vec)) : 
    a=feat_vec[i].copy()
    F.update(a)

print(type(F))
feat_vec2=F.copy()
print(type(feat_vec2))

'''
cnt_item = 0
for key, value in feat_vec2.items() :
    print (key, value)
    cnt_item = cnt_item +1

    if cnt_item == 30:
        break
'''

#e = 10
feat_vec = perc_train(train_data, tagset, 2)
#perc.perc_write_to_file(feat_vec, "default.model")
perc.perc_write_to_file(feat_vec2, "default.model")
print("wrote model to disk")

reading data ...
done.


<class 'collections.defaultdict'>
<class 'collections.defaultdict'>
epoch 0


In [2]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [9]:
str(output)

"Rockwell NNP B-NP\nInternational NNP B-NP\nCorp. NNP B-NP\n's POS B-NP\nTulsa NNP B-NP\nunit NN B-NP\nsaid VBD B-NP\nit PRP B-NP\nsigned VBD B-NP\na DT B-NP\ntentative JJ B-NP\nagreement NN B-NP\nextending VBG B-NP\nits PRP$ B-NP\ncontract NN B-NP\nwith IN B-NP\nBoeing NNP B-NP\nCo. NNP B-NP\nto TO B-NP\nprovide VB B-NP\nstructural JJ B-NP\nparts NNS B-NP\nfor IN B-NP\nBoeing NNP B-NP\n's POS B-NP\n747 CD B-NP\njetliners NNS B-NP\n. . B-NP\n\nRockwell NNP B-NP\nsaid VBD B-NP\nthe DT B-NP\nagreement NN B-NP\ncalls VBZ B-NP\nfor IN B-NP\nit PRP B-NP\nto TO B-NP\nsupply VB B-NP\n200 CD B-NP\nadditional JJ B-NP\nso-called JJ B-NP\nshipsets NNS B-NP\nfor IN B-NP\nthe DT B-NP\nplanes NNS B-NP\n. . B-NP\n\nThese DT B-NP\ninclude VBP B-NP\n, , B-NP\namong IN B-NP\nother JJ B-NP\nparts NNS B-NP\n, , B-NP\neach DT B-NP\njetliner NN B-NP\n's POS B-NP\ntwo CD B-NP\nmajor JJ B-NP\nbulkheads NNS B-NP\n, , B-NP\na DT B-NP\npressure NN B-NP\nfloor NN B-NP\n, , B-NP\ntorque NN B-NP\nbox NN B-NP\n, , B

In [3]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 11376; correct phrases: 906
             ADJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:     99
             ADVP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:    202
            CONJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:   7.96%; recall:  29.94%; F1:  12.58; found:  11376; correct:   3026
               PP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:   1221
              PRT: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:     22
             SBAR: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:    107
               VP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:   1100
accura

In [8]:
(test, _)

(defaultdict(list,
             {0: [('Rockwell', 'B-NP'),
               ('International', 'B-NP'),
               ('Corp.', 'B-NP'),
               ("'s", 'B-NP'),
               ('Tulsa', 'B-NP'),
               ('unit', 'B-NP'),
               ('said', 'B-NP'),
               ('it', 'B-NP'),
               ('signed', 'B-NP'),
               ('a', 'B-NP'),
               ('tentative', 'B-NP'),
               ('agreement', 'B-NP'),
               ('extending', 'B-NP'),
               ('its', 'B-NP'),
               ('contract', 'B-NP'),
               ('with', 'B-NP'),
               ('Boeing', 'B-NP'),
               ('Co.', 'B-NP'),
               ('to', 'B-NP'),
               ('provide', 'B-NP'),
               ('structural', 'B-NP'),
               ('parts', 'B-NP'),
               ('for', 'B-NP'),
               ('Boeing', 'B-NP'),
               ("'s", 'B-NP'),
               ('747', 'B-NP'),
               ('jetliners', 'B-NP'),
               ('.', 'B-NP'),
               ('

In [7]:
(reference, _)

(defaultdict(list,
             {0: [('Rockwell', 'B-NP'),
               ('International', 'I-NP'),
               ('Corp.', 'I-NP'),
               ("'s", 'B-NP'),
               ('Tulsa', 'I-NP'),
               ('unit', 'I-NP'),
               ('said', 'B-VP'),
               ('it', 'B-NP'),
               ('signed', 'B-VP'),
               ('a', 'B-NP'),
               ('tentative', 'I-NP'),
               ('agreement', 'I-NP'),
               ('extending', 'B-VP'),
               ('its', 'B-NP'),
               ('contract', 'I-NP'),
               ('with', 'B-PP'),
               ('Boeing', 'B-NP'),
               ('Co.', 'I-NP'),
               ('to', 'B-VP'),
               ('provide', 'I-VP'),
               ('structural', 'B-NP'),
               ('parts', 'I-NP'),
               ('for', 'B-PP'),
               ('Boeing', 'B-NP'),
               ("'s", 'B-NP'),
               ('747', 'I-NP'),
               ('jetliners', 'I-NP'),
               ('.', 'O')],
              1: [(